In [1]:
from gpstools import viz
from gpstools.parser import *
from gpstools.track import *
from gpstools.referencetrack import *
from gpstools.track_stats import *
import os
from collections import namedtuple

from gpstools.gps_utils import get_dist

In [2]:
tracks = load_tracks('data/rostov/')

Processing file data/rostov/rusabakumov_su3.gpx
Load track data/rostov/rusabakumov_su3.gpx with 6389 points
Looks that track has incorrect milliseconds presicion format (i.e written by Harry's Lap Timer). Fixing
Processing file data/rostov/rusabakumov_su2.gpx
Load track data/rostov/rusabakumov_su2.gpx with 6348 points
Looks that track has incorrect milliseconds presicion format (i.e written by Harry's Lap Timer). Fixing
Processing file data/rostov/rusabakumov_su1.gpx
Load track data/rostov/rusabakumov_su1.gpx with 6344 points
Looks that track has incorrect milliseconds presicion format (i.e written by Harry's Lap Timer). Fixing
Processing file data/rostov/sparrow_su3.csv
Looks like not a racebox track, trying racechrono format
407
Load racechrono track data/rostov/sparrow_su3.csv with 6977 points
Processing file data/rostov/maxmir.csv
Looks like not a racebox track, trying racechrono format
493
Load racechrono track data/rostov/maxmir.csv with 5406 points
Processing file data/rostov/fo

In [11]:
# Activity segments

START_RADIUS_KM = 0.02
ACTIVITY_DISTANCE_THRESHOLD_KM = 2.0

activity_start_points = {}

TrackWithActivitySegments = namedtuple('TrackWithActivitySegments', ['track', 'segments'])

def find_common_activity_segments(tracks):
    tracks_with_activity = []
    for track in tracks:
        tracks_with_activity.append(TrackWithActivitySegments(track, track.determine_activity_segments(allowed_pause = 5, segment_duration_threshold = 100)))
        
    for track in tracks_with_activity:
        for segment in track.segments:
            start_coords = segment.start_point.get_coords()
            if start_coords not in activity_start_points:
                activity_start_points[start_coords] = 0
                
            # Comparing with other coords
            for key in activity_start_points:
                if get_dist(start_coords, key) < START_RADIUS_KM:
                    activity_start_points[key] += 1
                
    most_popular_coord = None
    max_value = 0
    for key in activity_start_points:
        if activity_start_points[key] > max_value:
            most_popular_coord = key
            max_value = activity_start_points[key]
            
    assert most_popular_coord
    
    print("Found suitable start location at %s with %d segments" % (str(most_popular_coord), max_value))
    
    cropped_tracks = []
    for track in tracks_with_activity:
        segment_id = 1
        for segment in track.segments:
            start_coords = segment.start_point.get_coords()
            if get_dist(start_coords, most_popular_coord) < START_RADIUS_KM:
                cropped_tracks.append(
                    track.track.crop_track_to_segment(segment, track.track.name + "_" + str(segment_id))
                )
                segment_id +=1
                
    return cropped_tracks

def print_tracks_stats(tracks):
    print("Total %d tracks" % len(tracks))
    for i in range(len(tracks)):
        print("%d:" % i)
        tracks[i].print_stats()
                
cropped_tracks = find_common_activity_segments(tracks)  
print_tracks_stats(cropped_tracks)

Found suitable start location at Coordinate 56.9682, 39.1738 with 14 segments
Total 14 tracks
0:
Track rusabakumov_su3_1
Start time: 2019-01-27 11:27:02.900000
End time: 2019-01-27 11:36:14.100000
Total time: 0:09:11.200000
Total distance travelled: 11.313 km
Total average speed: 73.89 kph
Max speed: 147.82 kph
Average speed: 73.59 kph


1:
Track rusabakumov_su2_1
Start time: 2019-01-27 10:21:02.800000
End time: 2019-01-27 10:30:25.700000
Total time: 0:09:22.900000
Total distance travelled: 11.324 km
Total average speed: 72.42 kph
Max speed: 148.28 kph
Average speed: 72.11 kph


2:
Track rusabakumov_su1_1
Start time: 2019-01-27 08:28:02.500000
End time: 2019-01-27 08:37:40.400000
Total time: 0:09:37.900000
Total distance travelled: 11.328 km
Total average speed: 70.57 kph
Max speed: 145.01 kph
Average speed: 70.12 kph


3:
Track sparrow_su3_1
Start time: 2019-01-27 14:35:02
End time: 2019-01-27 14:45:13
Total time: 0:10:11
Total distance travelled: 11.472 km
Total average speed: 67.59 

In [14]:
# Reference test
from gpstools.trackcomparison import TrackComparison

ref_track = ReferenceTrack(cropped_tracks[2])
ref_track.print_stats()

comparison = TrackComparison(ref_track, cropped_tracks, 0.05)

comparison.build_comparison_graph('rostov_all', 'STI-Club Winter Cup Ростов')

# viz.plot_distance_speed_graph(
#     [rusabakumov_su2.dist_travelled, comparison._aligned_distances[3]],
#     [rusabakumov_su2.get_speed(), cheb_su2.get_speed(),]
# )

Reference track stats:
Total 1839 points
Distance between points (km):
Min: 0.00500, Max: 0.01975, Avg: 0.00614
Time difference between points (ms):
Min: 200000.00000, Max: 1300000.00000, Avg: 312513.60174
Total distance travelled: 11.282 km
Failed to find finish point for track elcamino_su3_1
Failed to find finish point for track yarik_su3_1
Done dist alignment for track rusabakumov_su3_1
Found 0 no candidates points
Found 3664 dist duplication points
Done dist alignment for track rusabakumov_su2_1
Found 0 no candidates points
Found 3706 dist duplication points
Done dist alignment for track rusabakumov_su1_1
Found 0 no candidates points
Found 3883 dist duplication points
Point with id 1128 has no distance found
Point with id 1129 has no distance found
Point with id 1130 has no distance found
Point with id 1131 has no distance found
Point with id 1132 has no distance found
Point with id 1133 has no distance found
Point with id 1134 has no distance found
Point with id 1135 has no distan

Done dist alignment for track elcamino_su3_1
Found 0 no candidates points
Found 7638 dist duplication points
Done dist alignment for track elcamino_su2_1
Found 0 no candidates points
Found 8266 dist duplication points
Done dist alignment for track elcamino_su1_1
Found 0 no candidates points
Found 8121 dist duplication points
Done dist alignment for track yarik_su1_1
Found 0 no candidates points
Found 2082 dist duplication points
Done dist alignment for track yarik_su3_1
Found 0 no candidates points
Found 552 dist duplication points
Done dist alignment for track yarik_su2_1
Found 0 no candidates points
Found 2137 dist duplication points


In [ ]:
# Reference viz
ref_as_track = Track('reference', ref_track.reference_points)

viz.plot_2d_track(ref_as_track)
ref_as_track.print_stats()